In [2]:
from vnstock import Company
from vnstock import Listing
from vnstock import Finance
company = Company(symbol = "VTO", source="TCBS")
company2 = Company(symbol = "YBG", source="VCI")
listing = Listing()
fin = finance = Finance(symbol="VTO", source="VCI")



In [3]:
fin.ratio().head()


Meta                         Chỉ tiêu cơ cấu nguồn vốn              \
  ticker yearReport lengthReport (ST+LT borrowings)/Equity Debt/Equity   
0    VTO       2025            2                  0.249357    0.361537   
1    VTO       2025            1                  0.264817    0.311995   
2    VTO       2024            4                  0.309917    0.380802   
3    VTO       2024            3                  0.344608    0.458407   
4    VTO       2024            2                  0.384278    0.432663   

                                                        \
  Fixed Asset-To-Equity Owners' Equity/Charter Capital   
0              0.387897                       1.393636   
1              0.391305                       1.489349   
2              0.434376                       1.465204   
3              0.510051                       1.422192   
4              0.548798                       1.397876   

  Chỉ tiêu hiệu quả hoạt động                                              \
               Asset Turnover Fixed Asset Turnover Days Sales Outstanding   
0                    0.715278             2.287768              20.915738   
1                    0.706117             2.097170              20.762411   
2                    0.686393             1.883759              24.310288   
3                    0.683574             1.773207              22.400156   
4                    0.654755             1.603855              23.876843   

   ... Chỉ tiêu thanh khoản        Chỉ tiêu định giá  \
   ...   Financial Leverage Market Capital (Bn. VND)   
0  ...             1.361537             954406658700   
1  ...             1.311995            1018299991500   
2  ...             1.380802             994339991700   
3  ...             1.458407            1102159990800   
4  ...             1.432663            1010313324900   

                                                                             \
  Outstanding Share (Mil. Shares)       P/E       P/B       P/S P/Cash Flow   
0                        79866666  9.185323  0.857469  0.840721    4.211909   
1                        79866666  9.920812  0.856079  0.896772    4.370360   
2                        79866666  8.971284  0.849711  0.888828    3.642677   
3                        79866666  9.318733  0.970333  0.987572    3.978533   
4                        79866666  8.909836  0.904944  0.946005    3.504457   

                                       
    EPS (VND)    BVPS (VND) EV/EBITDA  
0  386.266160  13936.359695  0.796538  
1  241.447114  14893.485387  1.052357  
2  430.113426  14652.038273  1.213413  
3  243.161798  14221.924846  1.592154  
4  370.454754  13978.763048  1.461058  

[5 rows x 37 columns]

In [5]:
import pandas as pd
df = pd.DataFrame(fin.income_statement())
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 2000)
print(df.head())

  ticker  yearReport  lengthReport  Revenue YoY (%)  Revenue (Bn. VND)  Attribute to parent company (Bn. VND)  Attribute to parent company YoY (%)  Financial Income  Interest Expenses         Sales     Net Sales  Cost of Sales  Gross Profit  Financial Expenses  Selling Expenses  General & Admin Expenses  Operating Profit/Loss  Other income  Net income from associated companies  Other Income/Expenses  Net other income/expenses  Profit before tax  Business income tax - current  Business income tax - deferred  Net Profit For the Year  Attributable to parent company
0    VTO        2025             2        -0.001131       258924368286                            30849790348                             0.042681        8652920238        -2879116310  258924368286  258924368286  -198850912075   60073456211         -3846273143                 0              -25313474721            39566628585      82566932                                     0             -196476547                 -113909615  

In [7]:
from vnstock.explorer.vci import Finance
fin = Finance(symbol='PLX', period='quarter')

In [15]:
fin._get_report(mode='raw').head()

,ticker,yearReport,lengthReport,updateDate,revenue,revenueGrowth,netProfit,netProfitGrowth,ebitMargin,roe,...,ISI97,CFA30,CFA2,CFB80,CFA38,CFA32,ISA5,BSA49,CFB64,__typename
0,PLX,2025,2,1756803601543,76568084039970,0.040544,1366405037475,0.133304,0.018088,0.082241,...,0.0,9.892070e+12,5.764672e+11,0.0,1.077064e+13,-1.620870e+12,5148108011024,3480171986189,0.0,CompanyFinancialRatio
1,PLX,2025,1,1746177217067,67890171339647,-0.096387,133414809920,-0.875659,0.001401,0.075706,...,0.0,-1.693769e+13,5.095880e+11,0.0,1.044673e+13,-1.007950e+07,3711902414777,3308339559036,0.0,CompanyFinancialRatio
2,PLX,2024,4,1743680658300,71055339774054,0.034548,545272943919,-0.149400,0.006667,0.111227,...,0.0,-1.142180e+13,6.737836e+11,0.0,1.493307e+13,-2.029978e+11,4639794814469,3339510809214,0.0,CompanyFinancialRatio
3,PLX,2024,3,1730314778847,64352149009035,-0.111638,65874458258,-0.910760,-0.002339,0.115055,...,0.0,-2.258806e+13,4.518609e+11,0.0,1.229352e+13,1.462638e+11,3436435356349,3023441914208,0.0,CompanyFinancialRatio
4,PLX,2024,2,1725440401193,73584636922071,0.118727,1205682247740,0.482842,0.016810,0.141182,...,0.0,-1.463131e+13,5.395484e+11,0.0,1.555723e+13,-2.050684e+12,4605641419515,2928188925989,0.0,CompanyFinancialRatio


In [ ]:

fin._get_ratio_dict()

,field_name,name,en_name,type,order,unit,com_type_code
0,BSA1,TỔNG TÀI SẢN NGẮN HẠN,CURRENT ASSETS,Chỉ tiêu cân đối kế toán,12,tỷ,BH
1,BSA2,Tiền và tương đương tiền,Cash and cash equivalents,Chỉ tiêu cân đối kế toán,13,tỷ,BH
2,BSA5,Giá trị thuần đầu tư ngắn hạn,Net Short-term investment,Chỉ tiêu cân đối kế toán,14,tỷ,BH
3,BSA8,Các khoản phải thu,Accounts Receivables,Chỉ tiêu cân đối kế toán,15,tỷ,BH
4,BSA15,Hàng tồn kho ròng,Net Inventories,Chỉ tiêu cân đối kế toán,16,tỷ,BH
...,...,...,...,...,...,...,...
733,ebit,EBIT (Tỷ đồng),EBIT (Bn. VND),Chỉ tiêu khả năng sinh lợi,6,tỷ,NH
734,dividend,Tỷ suất cổ tức (%),Dividend yield (%),Chỉ tiêu khả năng sinh lợi,13,%,NH
735,RTQ17,Vòng quay hàng tồn kho,Inventory Turnover,Chỉ tiêu hiệu quả hoạt động,28,index,NH
736,fae,TSCĐ / Vốn CSH,Fixed Asset-To-Equity,Chỉ tiêu cơ cấu nguồn vốn,36,index,NH
